# cert-manager und OpenSSL

In modernen Kubernetes-Umgebungen ist die sichere Kommunikation über TLS-Zertifikate essenziell. Zwei weit verbreitete Werkzeuge zur Verwaltung und Erstellung von Zertifikaten sind **cert-manager** und **OpenSSL**.  

#### **cert-manager**  
cert-manager ist eine leistungsstarke Kubernetes-Erweiterung zur automatisierten Verwaltung von TLS-Zertifikaten. Es ermöglicht die dynamische Ausstellung, Erneuerung und Überprüfung von Zertifikaten von verschiedenen **Certificate Authorities (CAs)** wie Let's Encrypt oder internen PKI-Systemen. Der Hauptvorteil von cert-manager liegt in seiner nahtlosen Integration mit Kubernetes **Ingress- und Secret-Ressourcen**, wodurch eine automatische TLS-Verschlüsselung für Dienste ermöglicht wird.  

#### **OpenSSL**  
OpenSSL ist eine universelle Open-Source-Bibliothek für kryptografische Operationen und die Erstellung von TLS/SSL-Zertifikaten. Es wird häufig für manuelle Zertifikatsanforderungen, die Generierung von **Certificate Signing Requests (CSRs)** und die Verwaltung von **Public Key Infrastructures (PKI)** verwendet. OpenSSL ist besonders nützlich für Entwicklungs- und Testzwecke sowie für Umgebungen, in denen eine vollständige Automatisierung über Kubernetes nicht erforderlich ist.  

Während **cert-manager** die Zertifikatsverwaltung in Kubernetes automatisiert, bleibt **OpenSSL** ein wertvolles Tool zur manuellen Zertifikatserstellung und -prüfung. Beide Werkzeuge ergänzen sich und sind essenziell für eine sichere Infrastruktur.

---

cert-manager steht als microk8s Addon zur Verfügung und muss nur enabled werden:

In [ ]:
%%bash
microk8s enable cert-manager

### CA für interne Zertifikate einrichten
Da es sich um eine interne DNS-Domain handelt, kann Let's Encrypt nicht genutzt werden. Stattdessen gibt es zwei Optionen:

Option 1: Self-Signed Issuer (einfach)
Falls du nur ein selbstsigniertes Zertifikat für interne Zwecke benötigst:

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: cert-manager.io/v1
kind: ClusterIssuer
metadata:
  name: selfsigned-cluster-issuer
spec:
  selfSigned: {}
EOF


### Zertifikatsanfrage für die interne Domain erstellen

Jetzt kann ein Zertifikat für die interne DNS-Domain (cna-cloud-hf-08-cp1.maas) erstellt werden:

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: cert-manager.io/v1
kind: Certificate
metadata:
  name: selfsigned-cert
  namespace: m01-at
spec:
  secretName: selfsigned-cert-secret
  duration: 24h
  renewBefore: 12h
  issuerRef:
    name: selfsigned-cluster-issuer
    kind: ClusterIssuer
  commonName: at.cloud-hf-08
  dnsNames:
    - at.cloud-hf-08
EOF

### Verwendung des Zertifikats

Das Zertifikat liegt nun als Kubernetes-Secret unter internal-cert-secret und kann z. B. für Ingress oder TLS-Services genutzt werden

In [ ]:
%%bash
kubectl delete -n m01-at ingress webshop
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: webshop 
  namespace: m01-at
  annotations:
    cert-manager.io/issuer: selfsigned-cluster-issuer
spec:
  rules:
  - host: at.cloud-hf-08
    http:
      paths:
      - path: /webshop
        pathType: Prefix
        backend:
          service:
            name: webshop
            port:
              number: 8080
  tls:
  - hosts:
    - at.cloud-hf-08
    secretName: selfsigned-cert-secret
EOF


Extrahieren der Zertifikate aus dem Secret und Ausgabe des Inhaltes

In [ ]:
%%bash
kubectl describe secret selfsigned-cert-secret -n m01-at 
kubectl get secret selfsigned-cert-secret -n m01-at -o jsonpath="{.data.ca\.crt}" | base64 --decode > ca.crt
kubectl get secret selfsigned-cert-secret -n m01-at -o jsonpath="{.data.tls\.crt}" | base64 --decode > tls.crt
kubectl get secret selfsigned-cert-secret -n m01-at -o jsonpath="{.data.tls\.key}" | base64 --decode > tls.key

In [ ]:
%%bash
openssl x509 -in ca.crt -text -noout

In [ ]:
%%bash
openssl x509 -in tls.crt -text -noout